In [14]:
import tkinter as tk
from tkinter import ttk, messagebox
import requests
import pandas as pd
from datetime import datetime, timedelta
import json
from urllib.parse import quote
import threading
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import os
import time

class InvestmentTableProgram:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("DAON_Searcher")

        # Matplotlib 한글 폰트 설정
        try:
            font_path = 'C:/Windows/Fonts/malgunbd.ttf'  # 맑은 고딕 볼드체
            if os.path.exists(font_path):
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = 'Malgun Gothic'
                plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 깨짐 방지
            else:
                messagebox.showwarning("폰트 경고", "맑은 고딕 폰트(malgunbd.ttf)를 찾을 수 없습니다. 차트에 한글이 깨질 수 있습니다.")
                plt.rcParams['font.family'] = 'DejaVu Sans'
        except Exception as e:
            messagebox.showwarning("폰트 설정 오류", f"폰트 설정 중 오류 발생: {e}")
            plt.rcParams['font.family'] = 'DejaVu Sans'

        self.root.state('zoomed')

        # 페이지 구성 관리
        self.page_order = [1, 2, 3, 4, 5]
        self.page_names = {
            1: "투자테이블",
            2: "시세차트",
            3: "페이지3",
            4: "페이지4",
            5: "페이지5"
        }
        self.current_page = 1

        # 필터 설정 변수들 (1페이지용)
        self.region_var = tk.StringVar(value="4481000000")
        self.sido_var = tk.StringVar()
        self.sigungu_var = tk.StringVar()
        self.region_data = {}  # 지역코드.txt에서 로드할 데이터

        self.start_date_var = tk.StringVar()
        self.end_date_var = tk.StringVar()
        end_date = datetime.now()
        start_date = end_date - pd.Timedelta(days=180)
        self.start_date_var.set(start_date.strftime("%Y-%m-%d"))
        self.end_date_var.set(end_date.strftime("%Y-%m-%d"))

        self.exclusive_area_min = tk.StringVar(value="0")
        self.exclusive_area_max = tk.StringVar(value="999")
        self.max_years = tk.StringVar(value="30")
        self.max_unit_price = tk.StringVar(value="999999")
        self.rate_min = tk.StringVar(value="-100")
        self.rate_max = tk.StringVar(value="100")

        # 데이터 저장 변수
        self.raw_data = None
        self.filtered_data = None
        self.selected_apartments = {}

        # 차트 관련 변수
        self.fig = None
        self.ax = None
        self.ax2 = None
        self.canvas = None
        self.chart_info_labels = {}
        self.current_chart_data = None
        self.current_selected_apt_info = None

        # 차트 옵션 변수
        self.show_sise_sale_var = tk.BooleanVar(value=True)
        self.show_sise_lease_var = tk.BooleanVar(value=True)
        self.show_sise_ratio_var = tk.BooleanVar(value=True)
        self.show_real_sale_var = tk.BooleanVar(value=True)
        self.show_real_lease_var = tk.BooleanVar(value=True)

        # 차트 기간 설정 변수 (2페이지용)
        self.chart_start_date_var = tk.StringVar()
        self.chart_end_date_var = tk.StringVar()

        # 전역 로딩 레이블
        self.loading_label = None

        # 실거래 리스트용 Treeviews
        self.sale_real_tran_tree = None
        self.lease_real_tran_tree = None

        # 네이버 부동산 매물 검색 관련 변수들
        self.naver_sido_map = {}
        self.naver_sigungu_map = {}
        self.naver_bjdong_map = {}
        self.naver_danji_map = {}
        self.naver_myeoneok_map = {}
        
        # 네이버 부동산 매물 검색 UI 변수들
        self.naver_sido_combobox = None
        self.naver_sigungu_combobox = None
        self.naver_bjdong_combobox = None
        self.naver_danji_combobox = None
        self.naver_myeoneok_combobox = None
        
        # 매물 목록 Treeview
        self.buy_article_tree = None
        self.rent_article_tree = None
        self.current_trade_type = tk.StringVar(value="매매")

        # 네이버 부동산 매물 데이터 저장 변수
        self.article_buy_data = None
        self.article_rent_data = None

        self.load_region_data()
        self.setup_ui()

    # User-Agent 기본 헤더 설정 (네이버 부동산용)
    DEFAULT_HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/json",
        "Referer": "https://new.land.naver.com/"
    }

    def fetch_naver_data(self, url):
        """네이버 부동산 API 데이터 호출 함수"""
        try:
            response = requests.get(url, headers=self.DEFAULT_HEADERS)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"네이버 API 호출 오류: {e}")
            return None

    def setup_ui(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.create_navigation_bar(main_frame)

        self.content_frame = ttk.Frame(main_frame)
        self.content_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        self.loading_label = ttk.Label(main_frame, text="프로그램 준비 완료", font=('Arial', 12))
        self.loading_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

        self.load_page(1)

    def create_navigation_bar(self, parent):
        nav_frame = ttk.Frame(parent)
        nav_frame.pack(fill=tk.X, pady=(0, 10))

        ttk.Button(nav_frame, text="페이지 순서 변경", command=self.change_page_order).pack(side=tk.LEFT, padx=(0, 20))

        self.page_buttons = {}
        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def change_page_order(self):
        dialog = tk.Toplevel(self.root)
        dialog.title("페이지 순서 변경")
        dialog.geometry("400x300")
        dialog.transient(self.root)
        dialog.grab_set()

        ttk.Label(dialog, text="페이지 순서를 변경하세요 (드래그 앤 드롭 또는 버튼 사용)").pack(pady=10)

        listbox = tk.Listbox(dialog, height=10)
        listbox.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        for i, page_num in enumerate(self.page_order):
            listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")

        btn_frame = ttk.Frame(dialog)
        btn_frame.pack(fill=tk.X, padx=20, pady=10)

        def move_up():
            selection = listbox.curselection()
            if selection and selection[0] > 0:
                idx = selection[0]
                self.page_order[idx], self.page_order[idx-1] = self.page_order[idx-1], self.page_order[idx]
                listbox.delete(0, tk.END)
                for i, page_num in enumerate(self.page_order):
                    listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")
                listbox.selection_set(idx+1)

        def move_down():
            selection = listbox.curselection()
            if selection and selection[0] < len(self.page_order) - 1:
                idx = selection[0]
                self.page_order[idx], self.page_order[idx+1] = self.page_order[idx+1], self.page_order[idx]
                listbox.delete(0, tk.END)
                for i, page_num in enumerate(self.page_order):
                    listbox.insert(tk.END, f"{i+1}. {self.page_names[page_num]}")
                listbox.selection_set(idx+1)

        def apply_changes():
            dialog.destroy()
            self.update_navigation_bar()

        ttk.Button(btn_frame, text="위로", command=move_up).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="아래로", command=move_down).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="적용", command=apply_changes).pack(side=tk.RIGHT, padx=5)
        ttk.Button(btn_frame, text="취소", command=dialog.destroy).pack(side=tk.RIGHT, padx=5)

    def update_navigation_bar(self):
        for btn in self.page_buttons.values():
            btn.destroy()

        if self.page_buttons:
            nav_frame = self.page_buttons[list(self.page_buttons.keys())[0]].master
        else:
            nav_frame_parent = self.root.winfo_children()[0]
            nav_frame = None
            for child in nav_frame_parent.winfo_children():
                if isinstance(child, ttk.Frame) and child.winfo_children():
                    nav_frame = child
                    break
            if not nav_frame:
                nav_frame = nav_frame_parent

        self.page_buttons = {}

        for i, page_num in enumerate(self.page_order):
            btn = ttk.Button(nav_frame, text=f"{i+1}. {self.page_names[page_num]}", command=lambda p=page_num: self.load_page(p))
            btn.pack(side=tk.LEFT, padx=5)
            self.page_buttons[page_num] = btn

    def load_page(self, page_num):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

        self.current_page = page_num

        if page_num == 1:
            self.create_investment_table_page()
        elif page_num == 2:
            self.create_chart_page()
        else:
            ttk.Label(self.content_frame, text=f"{self.page_names[page_num]} 페이지", font=('Arial', 20)).pack(pady=50)

    # ==================== 1페이지: 투자테이블 관련 메서드 ====================
    def create_investment_table_page(self):
        title_label = ttk.Label(self.content_frame, text="투자테이블", font=('Arial', 16, 'bold'))
        title_label.pack(pady=(0, 20))

        filter_frame = ttk.LabelFrame(self.content_frame, text="필터 설정", padding=10)
        filter_frame.pack(fill=tk.X, pady=(0, 10))

        self.create_filter_ui(filter_frame)

        result_frame = ttk.LabelFrame(self.content_frame, text="조회 결과", padding=10)
        result_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))

        self.create_result_table(result_frame)

    def create_filter_ui(self, parent):
        row = 0

        ttk.Label(parent, text="시도:").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        self.sido_combobox = ttk.Combobox(parent, textvariable=self.sido_var, values=list(self.region_data.keys()), state="readonly", width=15)
        self.sido_combobox.grid(row=row, column=1, padx=5, pady=5)
        self.sido_combobox.bind("<<ComboboxSelected>>", self.on_sido_selected)

        ttk.Label(parent, text="시군구:").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
        self.sigungu_combobox = ttk.Combobox(parent, textvariable=self.sigungu_var, state="readonly", width=15)
        self.sigungu_combobox.grid(row=row, column=3, padx=5, pady=5)

        if self.region_data:
            first_sido = list(self.region_data.keys())[0]
            self.sido_var.set(first_sido)
            self.on_sido_selected(None)
        else:
            self.sido_combobox.config(state="disabled")
            self.sigungu_combobox.config(state="disabled")
            messagebox.showwarning("경고", "지역코드 파일 로드에 실패하여 지역 선택 기능을 사용할 수 없습니다.")

        row += 1

        ttk.Label(parent, text="시작일 (YYYY-MM-DD):").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.start_date_var, width=15).grid(row=row, column=1, padx=5, pady=5)

        ttk.Label(parent, text="종료일 (YYYY-MM-DD):").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.end_date_var, width=15).grid(row=row, column=3, padx=5, pady=5)

        row += 1

        ttk.Label(parent, text="전용면적 (최소):").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.exclusive_area_min, width=15).grid(row=row, column=1, padx=5, pady=5)

        ttk.Label(parent, text="전용면적 (최대):").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.exclusive_area_max, width=15).grid(row=row, column=3, padx=5, pady=5)

        row += 1

        ttk.Label(parent, text="연차 (최대):").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.max_years, width=15).grid(row=row, column=1, padx=5, pady=5)

        ttk.Label(parent, text="최대 평단가:").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.max_unit_price, width=15).grid(row=row, column=3, padx=5, pady=5)

        row += 1

        ttk.Label(parent, text="증감율 (최소%):").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.rate_min, width=15).grid(row=row, column=1, padx=5, pady=5)

        ttk.Label(parent, text="증감율 (최대%):").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(parent, textvariable=self.rate_max, width=15).grid(row=row, column=3, padx=5, pady=5)

        row += 1

        ttk.Button(parent, text="조회", command=self.search_data).grid(row=row, column=0, columnspan=2, pady=10)
        ttk.Button(parent, text="Excel 내보내기", command=self.export_to_excel).grid(row=row, column=2, columnspan=2, pady=10)

    def on_sido_selected(self, event):
        selected_sido = self.sido_var.get()
        if selected_sido and selected_sido in self.region_data:
            sigungu_list = list(self.region_data[selected_sido].keys())
            self.sigungu_combobox['values'] = sigungu_list
            if sigungu_list:
                self.sigungu_var.set(sigungu_list[0])
            else:
                self.sigungu_var.set("")
        else:
            self.sigungu_combobox['values'] = []
            self.sigungu_var.set("")

    def load_region_data(self):
        file_path = r"C:\Users\kagaj\code\지역코드.txt"
        if not os.path.exists(file_path):
            messagebox.showerror("오류", f"지역코드 파일이 없습니다: {file_path}\n파일을 확인하거나 경로를 수정해주세요.")
            return

        self.region_data = {}
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    parts = line.split()
                    if len(parts) >= 3:
                        code = parts[0]
                        sido = parts[1]
                        sigungu = parts[2]
                        if sido not in self.region_data:
                            self.region_data[sido] = {}
                        self.region_data[sido][sigungu] = code
                    elif len(parts) == 2:
                        code = parts[0]
                        sido = parts[1]
                        if sido not in self.region_data:
                            self.region_data[sido] = {}
                        self.region_data[sido][''] = code
                    else:
                        pass  # print(f"경고: 지역코드 파일의 형식이 올바르지 않습니다: {line}")
        except Exception as e:
            messagebox.showerror("오류", f"지역코드 파일 로드 중 오류 발생: {str(e)}")
            self.region_data = {}        

    def create_result_table(self, parent):
        table_frame = ttk.Frame(parent)
        table_frame.pack(fill=tk.BOTH, expand=True)

        columns = ('지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수')
        self.tree = ttk.Treeview(table_frame, columns=columns, show='tree headings', height=20)

        self.tree.column("#0", width=30, minwidth=30, stretch=tk.NO)
        self.tree.heading("#0", text="", anchor=tk.CENTER)

        column_widths = {
            '지역': 120, '단지명': 150, '전용면적': 80, '연차': 60, '평단가': 100,
            '매매증감율': 100, '전세증감율': 100, '매매시세': 100, '전세시세': 100,
            '전세가율': 80, '매전갭': 80, '세대수': 80
        }

        self.sort_directions = {}

        for col in columns:
            self.tree.heading(col, text=col, command=lambda _col=col: self._sort_column(_col))
            self.tree.column(col, width=column_widths.get(col, 100), anchor=tk.CENTER)
            self.sort_directions[col] = False

        scrollbar_y = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.tree.yview)
        scrollbar_x = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.tree.xview)

        self.tree.configure(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

        self.tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)

        self.tree.bind("<<TreeviewSelect>>", self._on_tree_select)

    def _on_tree_select(self, event):
        selected_items_ids = self.tree.selection()
        temp_selected_apartments = {}
        for item_id in selected_items_ids:
            original_item_found = None
            for original_parsed_item in self.filtered_data:
                if str(original_parsed_item.get('면적일련번호')) == item_id:
                    original_item_found = original_parsed_item
                    break
            if original_item_found:
                temp_selected_apartments[item_id] = {
                    '지역': original_parsed_item.get('지역', ''),
                    '단지명': original_parsed_item.get('단지명', ''),
                    '전용면적': original_parsed_item.get('전용면적', ''),
                    '연차': original_parsed_item.get('연차', ''),
                    '면적일련번호': original_parsed_item.get('면적일련번호', ''),
                    '단지기본일련번호': original_parsed_item.get('단지기본일련번호', '')
                }
        self.selected_apartments = temp_selected_apartments
        if selected_items_ids:
            self.current_selected_apt_info = self.selected_apartments.get(selected_items_ids[-1])
        else:
            self.current_selected_apt_info = None

    def _sort_column(self, col):
        current_data = []
        for item_id in self.tree.get_children():
            current_data.append((self.tree.set(item_id, col), item_id))

        def get_sort_key(item_tuple):
            value = item_tuple[0]
            try:
                numeric_cols = ['전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수']
                if col in numeric_cols:
                    return float(value) if str(value).strip() else -float('inf')
                else:
                    return value
            except ValueError:
                return value

        reverse_sort = not self.sort_directions[col]
        self.sort_directions[col] = reverse_sort
        current_data.sort(key=get_sort_key, reverse=reverse_sort)
        for index, (value, item_id) in enumerate(current_data):
            self.tree.move(item_id, '', index)
        for c in self.tree['columns']:
            if c == col:
                self.tree.heading(c, text=f"{c} {'▼' if reverse_sort else '▲'}")
            else:
                self.tree.heading(c, text=c)

    def search_data(self):
        selected_sido = self.sido_var.get()
        selected_sigungu = self.sigungu_var.get()

        if not self.region_data:
            messagebox.showwarning("경고", "지역코드 파일이 로드되지 않아 지역 선택 기능을 사용할 수 없습니다.")
            self.loading_label.config(text="조회 실패: 지역 데이터 없음")
            return

        if selected_sido and selected_sido in self.region_data:
            if selected_sigungu in self.region_data[selected_sido]:
                self.region_var.set(self.region_data[selected_sido][selected_sigungu])
            elif '' in self.region_data[selected_sido] and not selected_sigungu:
                self.region_var.set(self.region_data[selected_sido][''])
            else:
                messagebox.showwarning("경고", "유효한 시군구를 선택해주세요.")
                self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
                return
        else:
            messagebox.showwarning("경고", "유효한 시도를 선택해주세요.")
            self.loading_label.config(text="조회 실패: 유효한 지역 선택 필요")
            return

        try:
            start_date_obj = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date_obj = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            if start_date_obj > end_date_obj:
                messagebox.showwarning("경고", "시작일은 종료일보다 빨라야 합니다.")
                self.loading_label.config(text="조회 실패: 날짜 범위 오류")
                return
        except ValueError:
            messagebox.showwarning("경고", "날짜 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            self.loading_label.config(text="조회 실패: 날짜 형식 오류")
            return

        self.loading_label.config(text="데이터를 조회 중입니다...")
        thread = threading.Thread(target=self.fetch_data)
        thread.daemon = True
        thread.start()

    def fetch_data(self):
        try:
            start_date_str = self.start_date_var.get()
            end_date_str = self.end_date_var.get()

            base_url = "https://api.kbland.kr/land-extra/price/v1/api/invstTblAptSearch"
            params = {
                "시작년월일": start_date_str.replace("-", ""),
                "종료년월일": end_date_str.replace("-", ""),
                "지역리스트": self.region_var.get()
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_table, data)

        except requests.exceptions.ConnectionError as e:
            error_msg = f"연결 오류: API 서버에 연결할 수 없습니다. 네트워크 상태를 확인하거나 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.Timeout as e:
            error_msg = f"시간 초과: 요청이 시간 초과되었습니다. 잠시 후 다시 시도해주세요."
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.HTTPError as e:
            error_msg = f"HTTP 오류: {e.response.status_code} - {e.response.reason}"
            self.root.after(0, self.show_error, error_msg)
        except ValueError as e:
            error_msg = f"데이터 형식 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)
        except Exception as e:
            error_msg = f"알 수 없는 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def parse_item_data(self, item):
        try:
            region = item.get('지역명', '')
            data_info = item.get('데이터정보', {})

            complex_name = data_info.get('I2010', {}).get('단지명', '')
            area_info = data_info.get('I2020', {})
            exclusive_area = area_info.get('전용면적', '')
            household_count = data_info.get('I2030', {}).get('지수', '')
            years = data_info.get('I2040', {}).get('지수', '')

            unit_price_info = data_info.get('I2050', {})
            unit_price = unit_price_info.get('공급면적당지수', '')
            if isinstance(unit_price, str):
                unit_price = unit_price.replace(',', '')

            sale_info = data_info.get('I2060', {})
            sale_price = sale_info.get('지수', '')
            sale_rate = sale_info.get('증감', '')
            lease_info = data_info.get('I2070', {})
            lease_price = lease_info.get('지수', '')
            lease_rate = lease_info.get('증감', '')
            lease_ratio_info = data_info.get('I2080', {})
            lease_ratio = lease_ratio_info.get('지수', '')

            gap_info = data_info.get('I2090', {})
            gap_value = gap_info.get('지수', '')

            area_serial_no = item.get('면적일련번호', '')
            complex_serial_no = item.get('단지기본일련번호', '')

            return {
                '지역': region, '단지명': complex_name, '전용면적': exclusive_area, '연차': years,
                '평단가': unit_price, '매매증감율': sale_rate, '전세증감율': lease_rate,
                '매매시세': sale_price, '전세시세': lease_price, '전세가율': lease_ratio,
                '매전갭': gap_value, '세대수': household_count,
                '면적일련번호': area_serial_no, '단지기본일련번호': complex_serial_no
            }
        except Exception as e:
            print(f"데이터 파싱 중 오류 발생: {e}, 원본 아이템: {item}")
            return None

    def update_table(self, data):
        for item in self.tree.get_children():
            self.tree.delete(item)

        self.raw_data = data
        parsed_data = []

        items_to_parse = []
        if isinstance(data, dict) and 'dataBody' in data and \
           isinstance(data['dataBody'], dict) and 'data' in data['dataBody'] and \
           isinstance(data['dataBody']['data'], dict) and '데이터목록' in data['dataBody']['data']:
            items_to_parse = data['dataBody']['data']['데이터목록']

        for item in items_to_parse:
            parsed_item = self.parse_item_data(item)
            if parsed_item:
                parsed_data.append(parsed_item)

        self.filtered_data = self.filter_data(parsed_data)

        for item in self.filtered_data:
            values = (
                item.get('지역', ''), item.get('단지명', ''), item.get('전용면적', ''),
                item.get('연차', ''), item.get('평단가', ''), item.get('매매증감율', ''),
                item.get('전세증감율', ''), item.get('매매시세', ''), item.get('전세시세', ''),
                item.get('전세가율', ''), item.get('매전갭', ''), item.get('세대수', '')
            )
            self.tree.insert('', tk.END, values=values, iid=str(item.get('면적일련번호', '')))

        self.loading_label.config(text=f"총 {len(self.filtered_data)}개의 결과를 조회했습니다.")
        if not self.filtered_data:
            self.loading_label.config(text="조회 조건에 맞는 데이터가 없습니다.")

    def filter_data(self, data):
        filtered = []
        for item in data:
            try:
                area_str = str(item.get('전용면적', '0')).strip()
                if area_str and area_str != '':
                    area = float(area_str)
                    if area < float(self.exclusive_area_min.get()) or area > float(self.exclusive_area_max.get()):
                        continue

                years_str = str(item.get('연차', '0')).strip()
                if years_str and years_str != '':
                    years = int(float(years_str))
                    if years > int(self.max_years.get()):
                        continue

                unit_price_str = str(item.get('평단가', '0')).strip()
                if unit_price_str and unit_price_str != '':
                    unit_price = float(unit_price_str)
                    if unit_price > float(self.max_unit_price.get()):
                        continue

                sale_rate_str = str(item.get('매매증감율', '0')).strip()
                if sale_rate_str and sale_rate_str != '':
                    sale_rate = float(sale_rate_str)
                    if sale_rate < float(self.rate_min.get()) or sale_rate > float(self.rate_max.get()):
                        continue
                filtered.append(item)
            except (ValueError, TypeError) as e:
                print(f"필터링 중 오류 발생: {e}, 데이터: {item}")
                filtered.append(item)
                continue
        return filtered

    def export_to_excel(self):
        if not hasattr(self, 'filtered_data') or not self.filtered_data:
            messagebox.showwarning("경고", "내보낼 데이터가 없습니다. 먼저 조회를 실행하세요.")
            return
        try:
            df = pd.DataFrame(self.filtered_data)
            columns_order = ['지역', '단지명', '전용면적', '연차', '평단가', '매매증감율', '전세증감율', '매매시세', '전세시세', '전세가율', '매전갭', '세대수', '면적일련번호', '단지기본일련번호']
            df = df.reindex(columns=columns_order, fill_value='')
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"투자테이블_{timestamp}.xlsx"
            df.to_excel(filename, index=False, engine='openpyxl')
            messagebox.showinfo("성공", f"데이터가 '{filename}' 파일로 내보내기 완료되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"Excel 내보내기 중 오류가 발생했습니다: {str(e)}")

    def show_error(self, error_msg):
        self.loading_label.config(text="오류가 발생했습니다.")
        messagebox.showerror("오류", error_msg)        

        # ==================== 2페이지: 시세차트 관련 메서드 ====================
    def create_chart_page(self):
        pane_window = ttk.PanedWindow(self.content_frame, orient=tk.HORIZONTAL)
        pane_window.pack(fill=tk.BOTH, expand=True)

        left_frame = ttk.Frame(pane_window, width=200, relief=tk.SUNKEN)
        pane_window.add(left_frame, weight=1)

        ttk.Label(left_frame, text="선택 단지 목록", font=('Arial', 14, 'bold')).pack(pady=10)

        selected_apt_columns = ('지역', '단지명', '전용면적', '연차')
        self.selected_apt_tree = ttk.Treeview(left_frame, columns=selected_apt_columns, show='headings', height=15)

        for col in selected_apt_columns:
            self.selected_apt_tree.heading(col, text=col, anchor=tk.CENTER)
            self.selected_apt_tree.column(col, width=80, anchor=tk.CENTER)

        selected_apt_scrollbar_y = ttk.Scrollbar(left_frame, orient=tk.VERTICAL, command=self.selected_apt_tree.yview)
        self.selected_apt_tree.configure(yscrollcommand=selected_apt_scrollbar_y.set)

        self.selected_apt_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        selected_apt_scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

        for item_id, apt_info in self.selected_apartments.items():
            self.selected_apt_tree.insert('', tk.END, values=(
                apt_info.get('지역', ''), apt_info.get('단지명', ''),
                apt_info.get('전용면적', ''), apt_info.get('연차', '')
            ), iid=item_id)

        self.selected_apt_tree.bind("<<TreeviewSelect>>", self.on_selected_apt_tree_select)

        # 중앙 프레임
        center_frame = ttk.Frame(pane_window, relief=tk.SUNKEN)
        pane_window.add(center_frame, weight=5)

        center_frame.grid_rowconfigure(0, weight=0)  # 옵션창 고정 높이
        center_frame.grid_rowconfigure(1, weight=6)  # 시세차트 (높이 비율 3에 해당)
        center_frame.grid_rowconfigure(2, weight=2)  # 거래건수 차트 (높이 비율 1에 해당)
        center_frame.grid_columnconfigure(0, weight=1)

        # 차트 옵션창
        chart_options_frame = ttk.LabelFrame(center_frame, text="차트 옵션", padding=10)
        chart_options_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

        # 차트 기간 설정 UI
        ttk.Label(chart_options_frame, text="차트 시작일:").grid(row=0, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_start_date_var, width=15).grid(row=0, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Label(chart_options_frame, text="차트 종료일:").grid(row=0, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Entry(chart_options_frame, textvariable=self.chart_end_date_var, width=15).grid(row=0, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Button(chart_options_frame, text="기간 적용", command=self.update_chart_view).grid(row=0, column=4, padx=10, pady=2)

        # 차트 표시 유형 선택 체크버튼
        ttk.Checkbutton(chart_options_frame, text="매매가", variable=self.show_sise_sale_var, command=self.update_chart_view).grid(row=1, column=0, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가", variable=self.show_sise_lease_var, command=self.update_chart_view).grid(row=1, column=1, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세가율", variable=self.show_sise_ratio_var, command=self.update_chart_view).grid(row=1, column=2, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="매매실거래", variable=self.show_real_sale_var, command=self.update_chart_view).grid(row=1, column=3, padx=5, pady=2, sticky=tk.W)
        ttk.Checkbutton(chart_options_frame, text="전세실거래", variable=self.show_real_lease_var, command=self.update_chart_view).grid(row=1, column=4, padx=5, pady=2, sticky=tk.W)

        # 시세차트 영역 (row=1)
        self.chart_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)

        self.fig = Figure(figsize=(6, 6), dpi=100)
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])  # 3:1 비율 설정
        self.ax = self.fig.add_subplot(gs[0, 0])  # 상단 시세차트
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)  # 하단 거래 건수 차트, x축 공유

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.chart_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # 차트 하단 주요 가격 정보 (row=2)
        self.chart_info_frame = ttk.Frame(center_frame, relief=tk.GROOVE, borderwidth=2)
        self.chart_info_frame.grid(row=2, column=0, sticky="nsew", padx=5, pady=5)

        # 3분할 프레임 및 레이블 생성
        price_info_frame1 = ttk.Frame(self.chart_info_frame)
        price_info_frame1.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        price_info_frame2 = ttk.Frame(self.chart_info_frame)
        price_info_frame2.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)
        price_info_frame3 = ttk.Frame(self.chart_info_frame)
        price_info_frame3.grid(row=0, column=2, sticky="nsew", padx=5, pady=5)
        
        self.chart_info_frame.grid_columnconfigure(0, weight=1)
        self.chart_info_frame.grid_columnconfigure(1, weight=1)
        self.chart_info_frame.grid_columnconfigure(2, weight=1)

        # 좌측 섹션: 매매 최고/최근가, 전세 최고/최근가
        ttk.Label(price_info_frame1, text="<시세 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매최고가'] = ttk.Label(price_info_frame1, text="매매최고가: -")
        self.chart_info_labels['매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['매매최근가'] = ttk.Label(price_info_frame1, text="매매최근가: -")
        self.chart_info_labels['매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최고가'] = ttk.Label(price_info_frame1, text="전세최고가: -")
        self.chart_info_labels['전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['전세최근가'] = ttk.Label(price_info_frame1, text="전세최근가: -")
        self.chart_info_labels['전세최근가'].pack(anchor=tk.W)

        # 가운데 섹션: 실거래 매매/전세 최고/최근가
        ttk.Label(price_info_frame2, text="<실거래 정보>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['실거래매매최고가'] = ttk.Label(price_info_frame2, text="실거래매매최고가: -")
        self.chart_info_labels['실거래매매최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래매매최근가'] = ttk.Label(price_info_frame2, text="실거래매매최근가: -")
        self.chart_info_labels['실거래매매최근가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최고가'] = ttk.Label(price_info_frame2, text="실거래전세최고가: -")
        self.chart_info_labels['실거래전세최고가'].pack(anchor=tk.W)
        self.chart_info_labels['실거래전세최근가'] = ttk.Label(price_info_frame2, text="실거래전세최근가: -")
        self.chart_info_labels['실거래전세최근가'].pack(anchor=tk.W)

        # 우측 섹션: 매매 하락가/율, 회복율
        ttk.Label(price_info_frame3, text="<매매 분석>", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=(0, 5))
        self.chart_info_labels['매매하락가'] = ttk.Label(price_info_frame3, text="매매하락가: -")
        self.chart_info_labels['매매하락가'].pack(anchor=tk.W)
        self.chart_info_labels['매매하락율'] = ttk.Label(price_info_frame3, text="매매하락율: -")
        self.chart_info_labels['매매하락율'].pack(anchor=tk.W)
        self.chart_info_labels['매매회복율'] = ttk.Label(price_info_frame3, text="매매회복율: -")
        self.chart_info_labels['매매회복율'].pack(anchor=tk.W)

        # 우측 프레임
        right_frame = ttk.PanedWindow(pane_window, orient=tk.VERTICAL)
        pane_window.add(right_frame, weight=2)

        # 우측 상단 - 실거래 정보창 (좌우 2분할)
        top_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(top_right_frame, weight=1)  # weight를 2로 변경 (40%)

        top_right_frame.grid_columnconfigure(0, weight=1)
        top_right_frame.grid_columnconfigure(1, weight=1)
        top_right_frame.grid_rowconfigure(0, weight=0)  # 라벨 행 고정
        top_right_frame.grid_rowconfigure(1, weight=1)  # Treeview 행 확장

        # 매매 실거래 Treeview (좌측 컬럼)
        sale_label_frame = ttk.Frame(top_right_frame)
        sale_label_frame.grid(row=0, column=0, sticky="ew", padx=2, pady=2)
        ttk.Label(sale_label_frame, text="매매 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        sale_tree_frame = ttk.Frame(top_right_frame)
        sale_tree_frame.grid(row=1, column=0, sticky="nsew", padx=2, pady=2)
        sale_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.sale_real_tran_tree = ttk.Treeview(sale_tree_frame, columns=sale_real_tran_cols, show='headings')  # height 제거
        for col in sale_real_tran_cols:
            self.sale_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.sale_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.sale_real_tran_tree.pack(fill=tk.BOTH, expand=True)  # expand=True로 부모 프레임 채움

        # 전세/월세 실거래 Treeview (우측 컬럼)
        lease_label_frame = ttk.Frame(top_right_frame)
        lease_label_frame.grid(row=0, column=1, sticky="ew", padx=2, pady=2)
        ttk.Label(lease_label_frame, text="전세/월세 실거래", font=('Arial', 10, 'bold')).pack(pady=(5, 0))

        lease_tree_frame = ttk.Frame(top_right_frame)
        lease_tree_frame.grid(row=1, column=1, sticky="nsew", padx=2, pady=2)
        lease_real_tran_cols = ('날짜', '실거래금액', '층수')
        self.lease_real_tran_tree = ttk.Treeview(lease_tree_frame, columns=lease_real_tran_cols, show='headings')  # height 제거
        for col in lease_real_tran_cols:
            self.lease_real_tran_tree.heading(col, text=col, anchor=tk.CENTER)
            self.lease_real_tran_tree.column(col, width=70, anchor=tk.CENTER)
        self.lease_real_tran_tree.pack(fill=tk.BOTH, expand=True)  # expand=True로 부모 프레임 채움

        # 우측 하단 - 네이버 부동산 매물 검색창
        bottom_right_frame = ttk.Frame(right_frame, relief=tk.GROOVE, borderwidth=2)
        right_frame.add(bottom_right_frame, weight=4)  # weight를 3으로 변경 (60%)
        
        self.create_naver_property_search(bottom_right_frame)

    # ==================== 네이버 부동산 매물 검색 기능 ====================
    def create_naver_property_search(self, parent):
        """네이버 부동산 매물 검색 UI 생성"""
        ttk.Label(parent, text="네이버 부동산 매물 검색", font=('Arial', 12, 'bold')).pack(pady=(5, 10))
        
        # 검색 조건 프레임
        search_frame = ttk.LabelFrame(parent, text="검색 조건", padding=5)
        search_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # 첫 번째 행: 시도, 시군구
        row1_frame = ttk.Frame(search_frame)
        row1_frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(row1_frame, text="시도:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_sido_combobox = ttk.Combobox(row1_frame, state="readonly", width=8)
        self.naver_sido_combobox.pack(side=tk.LEFT, padx=(0, 10))
        self.naver_sido_combobox.bind("<<ComboboxSelected>>", self.on_naver_sido_selected)
        
        ttk.Label(row1_frame, text="시군구:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_sigungu_combobox = ttk.Combobox(row1_frame, state="readonly", width=12)
        self.naver_sigungu_combobox.pack(side=tk.LEFT)
        self.naver_sigungu_combobox.bind("<<ComboboxSelected>>", self.on_naver_sigungu_selected)
        
        # 두 번째 행: 법정동, 단지
        row2_frame = ttk.Frame(search_frame)
        row2_frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(row2_frame, text="법정동:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_bjdong_combobox = ttk.Combobox(row2_frame, state="readonly", width=8)
        self.naver_bjdong_combobox.pack(side=tk.LEFT, padx=(0, 10))
        self.naver_bjdong_combobox.bind("<<ComboboxSelected>>", self.on_naver_bjdong_selected)
        
        ttk.Label(row2_frame, text="단지:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_danji_combobox = ttk.Combobox(row2_frame, state="readonly", width=20)
        self.naver_danji_combobox.pack(side=tk.LEFT)
        self.naver_danji_combobox.bind("<<ComboboxSelected>>", self.on_naver_danji_selected)
        
        # 세 번째 행: 면적, 거래유형, 검색버튼
        row3_frame = ttk.Frame(search_frame)
        row3_frame.pack(fill=tk.X, pady=2)
        
        ttk.Label(row3_frame, text="면적:").pack(side=tk.LEFT, padx=(0, 5))
        self.naver_myeoneok_combobox = ttk.Combobox(row3_frame, state="readonly", width=12)
        self.naver_myeoneok_combobox.pack(side=tk.LEFT, padx=(0, 10))
        ttk.Label(row3_frame, text="거래유형:").pack(side=tk.LEFT, padx=(0, 5))
        trade_type_frame = ttk.Frame(row3_frame)
        trade_type_frame.pack(side=tk.LEFT, padx=(0, 10))
        
        ttk.Radiobutton(trade_type_frame, text="매매", variable=self.current_trade_type, value="매매").pack(side=tk.LEFT)
        ttk.Radiobutton(trade_type_frame, text="전세", variable=self.current_trade_type, value="전세").pack(side=tk.LEFT, padx=(10, 0))
        
        ttk.Button(row3_frame, text="매물 검색", command=self.search_naver_properties).pack(side=tk.LEFT, padx=(10, 0))
        
        # 매물 목록 프레임
        property_list_frame = ttk.LabelFrame(parent, text="매물 목록", padding=5)
        property_list_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # 매물 목록 Treeview
        property_columns = ('가격', '동', '층', '면적')
        self.property_tree = ttk.Treeview(property_list_frame, columns=property_columns, show='headings', height=10)
        
        for col in property_columns:
            self.property_tree.heading(col, text=col, anchor=tk.CENTER)
            if col == '가격':
                self.property_tree.column(col, width=120, anchor=tk.E)
            elif col == '동':
                self.property_tree.column(col, width=80, anchor=tk.CENTER)
            elif col == '층':
                self.property_tree.column(col, width=60, anchor=tk.CENTER)
            elif col == '면적':
                self.property_tree.column(col, width=80, anchor=tk.CENTER)
        
        # 스크롤바
        property_scrollbar = ttk.Scrollbar(property_list_frame, orient=tk.VERTICAL, command=self.property_tree.yview)
        self.property_tree.configure(yscrollcommand=property_scrollbar.set)
        
        self.property_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        property_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # 초기 데이터 로드
        self.load_naver_sido_list()

    def load_naver_sido_list(self):
        """네이버 부동산 시도 목록 로드"""
        url = "https://m.land.naver.com/map/getRegionList?cortarNo=0000000000&mycortarNo="
        data = self.fetch_naver_data(url)
        if data and 'result' in data and 'list' in data['result']:
            sido_list_raw = data['result']['list']
            self.naver_sido_map = {item['CortarNm']: item['CortarNo'] for item in sido_list_raw}
            self.naver_sido_combobox['values'] = list(self.naver_sido_map.keys())
        else:
            print("네이버 시도 목록 로드 실패")
            self.naver_sido_combobox['values'] = ["목록 로드 실패"]

    def on_naver_sido_selected(self, event):
        """네이버 시도 선택 시 시군구 목록 로드"""
        selected_sido_name = self.naver_sido_combobox.get()
        selected_sido_code = self.naver_sido_map.get(selected_sido_name)
        if selected_sido_code:
            self.load_naver_sigungu_list(selected_sido_code)
            self.clear_naver_lower_combos(['sigungu', 'bjdong', 'danji', 'myeoneok'])

    def load_naver_sigungu_list(self, sido_code):
        """네이버 부동산 시군구 목록 로드"""
        url = f"https://m.land.naver.com/map/getRegionList?cortarNo={sido_code}&mycortarNo={sido_code}"
        data = self.fetch_naver_data(url)
        if data and 'result' in data and 'list' in data['result']:
            sigungu_list_raw = data['result']['list']
            self.naver_sigungu_map = {item['CortarNm']: item['CortarNo'] for item in sigungu_list_raw}
            self.naver_sigungu_combobox['values'] = list(self.naver_sigungu_map.keys())
        else:
            print(f"네이버 시군구 목록 로드 실패 (시도 코드: {sido_code})")
            self.naver_sigungu_combobox['values'] = ["목록 로드 실패"]

    def on_naver_sigungu_selected(self, event):
        """네이버 시군구 선택 시 법정동 목록 로드"""
        selected_sigungu_name = self.naver_sigungu_combobox.get()
        selected_sigungu_code = self.naver_sigungu_map.get(selected_sigungu_name)
        if selected_sigungu_code:
            self.load_naver_bjdong_list(selected_sigungu_code)
            self.clear_naver_lower_combos(['bjdong', 'danji', 'myeoneok'])

    def load_naver_bjdong_list(self, sigungu_code):
        """네이버 부동산 법정동 목록 로드"""
        url = f"https://m.land.naver.com/map/getRegionList?cortarNo={sigungu_code}&mycortarNo={sigungu_code}"
        data = self.fetch_naver_data(url)
        if data and 'result' in data and 'list' in data['result']:
            bjdong_list_raw = data['result']['list']
            self.naver_bjdong_map = {item['CortarNm']: item['CortarNo'] for item in bjdong_list_raw}
            self.naver_bjdong_combobox['values'] = list(self.naver_bjdong_map.keys())
        else:
            print(f"네이버 법정동 목록 로드 실패 (시군구 코드: {sigungu_code})")
            self.naver_bjdong_combobox['values'] = ["목록 로드 실패"]

    def on_naver_bjdong_selected(self, event):
        """네이버 법정동 선택 시 단지 목록 로드"""
        selected_bjdong_name = self.naver_bjdong_combobox.get()
        selected_bjdong_code = self.naver_bjdong_map.get(selected_bjdong_name)
        if selected_bjdong_code:
            self.load_naver_danji_list(selected_bjdong_code)
            self.clear_naver_lower_combos(['danji', 'myeoneok'])

    def load_naver_danji_list(self, bjdong_code):
        """네이버 부동산 단지 목록 로드"""
        url = f"https://m.land.naver.com/complex/ajax/complexListByCortarNo?cortarNo={bjdong_code}"
        data = self.fetch_naver_data(url)
        if data and 'result' in data and isinstance(data['result'], list):
            danji_list_raw = data['result']
            self.naver_danji_map = {}
            danji_display_list = []

            for item in danji_list_raw:
                hscp_no = item.get('hscpNo')
                hscp_nm = item.get('hscpNm')
                hscp_type_nm = item.get('hscpTypeNm')

                if hscp_no and hscp_nm and hscp_type_nm:
                    display_text = f"{hscp_nm}({hscp_type_nm})"
                    self.naver_danji_map[display_text] = hscp_no
                    danji_display_list.append(display_text)

            self.naver_danji_combobox['values'] = danji_display_list
        else:
            print(f"네이버 단지 목록 로드 실패 (법정동 코드: {bjdong_code})")
            self.naver_danji_combobox['values'] = ["목록 로드 실패"]

    def on_naver_danji_selected(self, event):
        """네이버 단지 선택 시 면적 목록 로드"""
        selected_danji_name = self.naver_danji_combobox.get()
        selected_danji_code = self.naver_danji_map.get(selected_danji_name)
        if selected_danji_code:
            self.load_naver_myeoneok_list(selected_danji_code)

    def load_naver_myeoneok_list(self, danji_code):
        """네이버 부동산 면적 목록 로드"""
        url = f"https://fin.land.naver.com/front-api/v1/complex/pyeongList?complexNumber={danji_code}"
        data = self.fetch_naver_data(url)
        
        if data and 'result' in data and isinstance(data['result'], list):
            myeoneok_list_raw = data['result']
            self.naver_myeoneok_map = {}
            myeoneok_display_list = []

            for item in myeoneok_list_raw:
                pyeong_type_no = item.get('number')
                supply_area = item.get('supplyArea')
                exclusive_area = item.get('exclusiveArea')
                name_type = item.get('nameType', '')

                if pyeong_type_no is not None and supply_area is not None and exclusive_area is not None:
                    display_text = f"{supply_area:.2f}({exclusive_area:.2f}{name_type})"
                    self.naver_myeoneok_map[display_text] = pyeong_type_no
                    myeoneok_display_list.append(display_text)

            self.naver_myeoneok_combobox['values'] = myeoneok_display_list
        else:
            print(f"네이버 면적 목록 로드 실패 (단지 코드: {danji_code})")
            self.naver_myeoneok_combobox['values'] = ["목록 로드 실패"]

    def clear_naver_lower_combos(self, combo_list):
        """하위 콤보박스들 초기화"""
        for combo in combo_list:
            if combo == 'sigungu':
                self.naver_sigungu_combobox.set('')
                self.naver_sigungu_combobox['values'] = []
            elif combo == 'bjdong':
                self.naver_bjdong_combobox.set('')
                self.naver_bjdong_combobox['values'] = []
            elif combo == 'danji':
                self.naver_danji_combobox.set('')
                self.naver_danji_combobox['values'] = []
            elif combo == 'myeoneok':
                self.naver_myeoneok_combobox.set('')
                self.naver_myeoneok_combobox['values'] = []
        
        # 매물 목록도 초기화
        if hasattr(self, 'property_tree'):
            for item in self.property_tree.get_children():
                self.property_tree.delete(item)

    def search_naver_properties(self):
        """네이버 부동산 매물 검색 실행"""
        selected_danji_name = self.naver_danji_combobox.get()
        selected_myeoneok_name = self.naver_myeoneok_combobox.get()
        selected_danji_code = self.naver_danji_map.get(selected_danji_name)
        selected_myeoneok_code = self.naver_myeoneok_map.get(selected_myeoneok_name)
        
        if not selected_danji_code or not selected_myeoneok_code:
            messagebox.showwarning("경고", "단지와 면적을 모두 선택해주세요.")
            return
        
        trade_type = self.current_trade_type.get()
        trade_type_code = "A1" if trade_type == "매매" else "B1"
        
        self.loading_label.config(text=f"{trade_type} 매물을 검색 중입니다...")
        
        # 별도 스레드에서 매물 검색 실행
        thread = threading.Thread(target=self.fetch_naver_properties, args=(selected_danji_code, selected_myeoneok_code, trade_type_code, trade_type))
        thread.daemon = True
        thread.start()

    def fetch_naver_properties(self, danji_code, myeoneok_code, trade_type_code, trade_type_name):
        """네이버 부동산 매물 데이터 가져오기"""
        try:
            url = f"https://fin.land.naver.com/front-api/v1/complex/article/list?complexNumber={danji_code}&pyeongTypeNumbers%5B%5D={myeoneok_code}&tradeTypes%5B%5D={trade_type_code}&dateDescending=false&userChannelType=PC&page=0"
            data = self.fetch_naver_data(url)
            
            if data and 'result' in data:
                article_data = data['result']
                self.root.after(0, self.update_property_list, article_data, trade_type_name)
            else:
                self.root.after(0, self.show_property_search_error, "매물 데이터를 가져올 수 없습니다.")
                
        except Exception as e:
            error_msg = f"매물 검색 중 오류 발생: {str(e)}"
            self.root.after(0, self.show_property_search_error, error_msg)

    def update_property_list(self, article_data, trade_type_name):
        """매물 목록 업데이트"""
        # 기존 목록 삭제
        for item in self.property_tree.get_children():
            self.property_tree.delete(item)
        
        article_list = article_data.get('list', [])
        total_count = article_data.get('totalCount', 0)
        
        if not article_list:
            self.loading_label.config(text=f"{trade_type_name} 매물이 없습니다.")
            return
        
        # 매물 목록 추가
        for item in article_list:
            try:
                rep_info = item.get('representativeArticleInfo', {})
                price_info = rep_info.get('priceInfo', {})
                article_detail = rep_info.get('articleDetail', {})
                
                # 가격 정보 추출
                if trade_type_name == "매매":
                    price = price_info.get('dealPrice', 0)
                    price_text = f"{price:,}만원" if price else "-"
                else:  # 전세
                    price = price_info.get('warrantyPrice', 0)
                    price_text = f"{price:,}만원" if price else "-"
                
                # 기타 정보 추출
                dong_name = rep_info.get('dongName', '-')
                floor_info = article_detail.get('floorInfo', '-')
                
                # 면적 정보 (이미 선택된 면적이므로 콤보박스에서 가져옴)
                area_text = self.naver_myeoneok_combobox.get().split('(')[0] if self.naver_myeoneok_combobox.get() else '-'
                
                # Treeview에 추가
                self.property_tree.insert('', tk.END, values=(price_text, dong_name, floor_info, area_text + "㎡"))
                
            except Exception as e:
                print(f"매물 정보 파싱 중 오류: {e}")
                continue
        
        self.loading_label.config(text=f"{trade_type_name} 매물 {len(article_list)}개를 찾았습니다. (총 {total_count}개)")

    def show_property_search_error(self, error_msg):
        """매물 검색 오류 표시"""
        self.loading_label.config(text="매물 검색 실패")
        messagebox.showerror("매물 검색 오류", error_msg)

# ==================== 기존 차트 관련 메서드들 ====================
    def on_selected_apt_tree_select(self, event):
        selected_item_id = self.selected_apt_tree.focus()
        if selected_item_id and selected_item_id in self.selected_apartments:
            apt_info = self.selected_apartments[selected_item_id]
            complex_serial_no = apt_info.get('단지기본일련번호')
            area_serial_no = apt_info.get('면적일련번호')
            self.current_selected_apt_info = apt_info
            if complex_serial_no and area_serial_no:
                self.loading_label.config(text=f"'{apt_info.get('단지명')} {apt_info.get('전용면적')}㎡' 시세 데이터를 조회 중입니다...")
                thread_chart = threading.Thread(target=self.fetch_chart_data, args=(complex_serial_no, area_serial_no))
                thread_chart.daemon = True
                thread_chart.start()
                thread_real_tran = threading.Thread(target=self.fetch_real_transaction_data, args=(complex_serial_no, area_serial_no))
                thread_real_tran.daemon = True
                thread_real_tran.start()
            else:
                messagebox.showwarning("경고", "선택된 단지의 단지기본일련번호 또는 면적일련번호가 없습니다.")
                self.loading_label.config(text="데이터 조회 실패: 필수 정보 누락")
        else:
            self.loading_label.config(text="단지를 선택해주세요.")
            self.current_selected_apt_info = None

    def fetch_chart_data(self, complex_serial_no, area_serial_no):
        try:
            end_date = datetime.now()
            start_date_api = "19700101"
            end_date_api = end_date.strftime("%Y%m%d")

            base_url = "https://api.kbland.kr/land-price/price/PerMn/IntgrationChart"
            params = {
                "단지기본일련번호": complex_serial_no, "면적일련번호": area_serial_no,
                "거래구분": "0", "조회구분": "2",
                "조회시작일": start_date_api, "조회종료일": end_date_api
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError(f"JSON 응답이 아닙니다. Content-Type: {response.headers.get('content-type')}")

            self.root.after(0, self.update_chart_display, data)

        except requests.exceptions.ConnectionError as e:
            error_msg = f"Chart API 연결 오류: {e}"
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.Timeout as e:
            error_msg = f"Chart API 시간 초과: {e}"
            self.root.after(0, self.show_error, error_msg)
        except requests.exceptions.HTTPError as e:
            error_msg = f"Chart API HTTP 오류: {e.response.status_code} - {e.response.reason}"
            self.root.after(0, self.show_error, error_msg)
        except ValueError as e:
            error_msg = f"Chart API 데이터 형식 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)
        except Exception as e:
            error_msg = f"Chart API 알 수 없는 오류: {str(e)}"
            self.root.after(0, self.show_error, error_msg)

    def fetch_real_transaction_data(self, complex_serial_no, area_serial_no):
        """실거래 정보 API에서 데이터 가져오기"""
        try:
            base_url = "https://api.kbland.kr/land-price/price/LatestRealTranPrc"
            params = {
                "단지기본일련번호": complex_serial_no,
                "면적일련번호": area_serial_no,
                "거래구분": "0",  # 0: 전체
                "조회구분": "0",  # 0: 전체
                "첫페이지갯수": "100000",  # 충분히 큰 값으로 모든 데이터 가져오기 시도
                "현재페이지": "1",
                "페이지갯수": "0"  # 이 값은 API 문서에 따라 다를 수 있음 (0이 전체를 의미하거나, 무시되거나)
            }
            url = base_url + "?" + "&".join([f"{k}={quote(str(v))}" for k, v in params.items()])

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'application/json, text/plain, */*',
                'Accept-Language': 'ko-KR,ko;q=0.9,en;q=0.8',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
                'Referer': 'https://kbland.kr/',
                'Origin': 'https://kbland.kr'
            }
            session = requests.Session()
            session.headers.update(headers)

            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = session.get(url, timeout=30)
                    response.raise_for_status()
                    break
                except requests.exceptions.ConnectionError as e:
                    if attempt == max_retries - 1:
                        raise e
                    time.sleep(2)

            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                raise ValueError("실거래 API JSON 응답이 아닙니다.")
            
            sale_list, lease_list = self.parse_real_transaction_data(data)
            self.root.after(0, self.display_real_transaction_data, sale_list, lease_list)

        except Exception as e:
            self.root.after(0, self.show_error, f"실거래 API 오류: {e}")

    def parse_real_transaction_data(self, raw_data):
        """실거래 정보 API 응답 데이터 파싱 및 분류"""
        sale_list = []
        lease_list = []
        items = raw_data.get('dataBody', {}).get('data', []) 
        
        for item in items:
            물건거래명 = item.get('물건거래명', '')
            계약년월 = item.get('계약년월', '')
            계약일 = item.get('계약일', '')
            해당층수 = item.get('해당층수', '-')
            
            날짜 = f"{계약년월[:4]}-{계약년월[4:]}-{계약일}" if 계약년월 and 계약일 else '-'
            
            거래금액_표시 = '-'
            if 물건거래명 == '매매':
                금액_str = str(item.get('매매실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                sale_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '전세':
                금액_str = str(item.get('전세실거래금액', '0')).replace(',', '')
                거래금액_표시 = f"{float(금액_str):.0f} 만원" if 금액_str and float(금액_str) > 0 else '-'
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            elif 물건거래명 == '월세':
                보증금액_str = str(item.get('보증금액', '0')).replace(',', '')
                월세금액_str = str(item.get('월세금액', '0')).replace(',', '')
                보증금 = f"{float(보증금액_str):.0f}" if 보증금액_str and float(보증금액_str) > 0 else '0'
                월세 = f"{float(월세금액_str):.0f}" if 월세금액_str and float(월세금액_str) > 0 else '0'
                거래금액_표시 = f"{보증금}/{월세} 만원"
                lease_list.append((날짜, 거래금액_표시, 해당층수))
            
        sale_list.sort(key=lambda x: x[0], reverse=True)
        lease_list.sort(key=lambda x: x[0], reverse=True)

        return sale_list, lease_list

    def display_real_transaction_data(self, sale_list, lease_list):
        """실거래 정보를 Treeview에 표시"""
        # 기존 데이터 삭제
        for item in self.sale_real_tran_tree.get_children():
            self.sale_real_tran_tree.delete(item)
        for item in self.lease_real_tran_tree.get_children():
            self.lease_real_tran_tree.delete(item)
        
        # 새 데이터 삽입
        for row_data in sale_list:
            self.sale_real_tran_tree.insert('', tk.END, values=row_data)
        for row_data in lease_list:
            self.lease_real_tran_tree.insert('', tk.END, values=row_data)

    def update_chart_display(self, chart_data_raw):
        parsed_chart_data = self.parse_chart_api_data(chart_data_raw)
        self.current_chart_data = parsed_chart_data

        if parsed_chart_data and (parsed_chart_data['sise_data'] or \
           parsed_chart_data['sale_real_data'] or parsed_chart_data['lease_real_data']):

            all_dates_in_chart_data = []
            if parsed_chart_data['sise_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in parsed_chart_data['sise_data']])
            if parsed_chart_data['sale_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['sale_real_data'] if d['거래금액'] is not None])
            if parsed_chart_data['lease_real_data']:
                all_dates_in_chart_data.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in parsed_chart_data['lease_real_data'] if d['거래금액'] is not None])

            if all_dates_in_chart_data:
                min_date = min(all_dates_in_chart_data)
                max_date = max(all_dates_in_chart_data)
                self.chart_start_date_var.set(min_date.strftime("%Y-%m-%d"))
                self.chart_end_date_var.set(max_date.strftime("%Y-%m-%d"))
            else:
                self.chart_start_date_var.set("")
                self.chart_end_date_var.set("")

            self.update_chart_view()
            self.loading_label.config(text="시세 차트가 업데이트되었습니다.")
        else:
            self.fig.clf()
            gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
            self.ax = self.fig.add_subplot(gs[0, 0])
            self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            self.update_chart_summary_info(None)
            self.loading_label.config(text="선택된 단지의 시세 데이터를 찾을 수 없습니다.")
            self.chart_start_date_var.set("")
            self.chart_end_date_var.set("")

    def update_chart_view(self):
        if not self.current_chart_data:
            self.loading_label.config(text="차트 데이터가 없습니다.")
            return

        filtered_sise_data = []
        filtered_sale_real_data = []
        filtered_lease_real_data = []

        try:
            chart_start_date = datetime.strptime(self.chart_start_date_var.get(), "%Y-%m-%d").date()
            chart_end_date = datetime.strptime(self.chart_end_date_var.get(), "%Y-%m-%d").date()
            if chart_start_date > chart_end_date:
                messagebox.showwarning("경고", "차트 시작일은 종료일보다 빨라야 합니다.")
                return
        except ValueError:
            messagebox.showwarning("경고", "차트 기간 형식이 올바르지 않습니다 (YYYY-MM-DD).")
            return

        for item in self.current_chart_data['sise_data']:
            item_date = datetime.strptime(item['기준년월'], "%Y%m").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sise_data.append(item)

        for item in self.current_chart_data['sale_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_sale_real_data.append(item)

        for item in self.current_chart_data['lease_real_data']:
            item_date = datetime.strptime(item['기준년월일'], "%Y%m%d").date()
            if chart_start_date <= item_date <= chart_end_date:
                filtered_lease_real_data.append(item)

        self.draw_price_chart(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data},
            show_sise_sale=self.show_sise_sale_var.get(),
            show_sise_lease=self.show_sise_lease_var.get(),
            show_sise_ratio=self.show_sise_ratio_var.get(),
            show_real_sale=self.show_real_sale_var.get(),
            show_real_lease=self.show_real_lease_var.get(),
            apt_info=self.current_selected_apt_info  # 단지 정보 전달
        )
        self.update_chart_summary_info(
            {'sise_data': filtered_sise_data, 'sale_real_data': filtered_sale_real_data, 'lease_real_data': filtered_lease_real_data}
        )

    def parse_chart_api_data(self, chart_data_raw):
        sise_data = []
        sale_real_data = []
        lease_real_data = []
        try:
            data_body_data = chart_data_raw.get('dataBody', {}).get('data', {})

            sise_groups = data_body_data.get('시세', [])
            for group in sise_groups:
                if 'items' in group and isinstance(group['items'], list):
                    for item in group['items']:
                        sise_data.append({
                            '기준년월': item.get('기준년월', ''),
                            '매매일반거래가': item.get('매매일반거래가', None),
                            '전세일반거래가': item.get('전세일반거래가', None),
                            '전세가율': item.get('전세가율', None)
                        })
                        if '매매실거래금액리스트' in item and isinstance(item['매매실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['매매실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    sale_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
                        if '전세실거래금액리스트' in item and isinstance(item['전세실거래금액리스트'], list):
                            기준년월 = item.get('기준년월', '')
                            for 금액_str in item['전세실거래금액리스트']:
                                거래금액 = float(str(금액_str).replace(',', '')) if str(금액_str).replace(',', '') else None
                                if 거래금액 is not None:
                                    lease_real_data.append({'기준년월일': 기준년월 + '01', '거래금액': 거래금액})
            sise_data.sort(key=lambda x: x['기준년월'])

            sale_real_df = pd.DataFrame(sale_real_data)
            if not sale_real_df.empty:
                sale_real_data = sale_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')
            lease_real_df = pd.DataFrame(lease_real_data)
            if not lease_real_df.empty:
                lease_real_data = lease_real_df.drop_duplicates(subset=['기준년월일', '거래금액']).to_dict(orient='records')

            sale_real_data.sort(key=lambda x: x['기준년월일'])
            lease_real_data.sort(key=lambda x: x['기준년월일'])

            return {
                'sise_data': sise_data,
                'sale_real_data': sale_real_data,
                'lease_real_data': lease_real_data
            }
        except Exception as e:
            self.root.after(0, self.show_error, f"차트 데이터 파싱 오류: {e}")
            return None

    def draw_price_chart(self, chart_data, show_sise_sale=True, show_sise_lease=True, show_sise_ratio=True, show_real_sale=True, show_real_lease=True, apt_info=None):
        self.fig.clf()
        gs = self.fig.add_gridspec(nrows=2, ncols=1, height_ratios=[3, 1])
        self.ax = self.fig.add_subplot(gs[0, 0])
        self.ax2 = self.fig.add_subplot(gs[1, 0], sharex=self.ax)

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        if not (sise_data or sale_real_data or lease_real_data):
            self.ax.text(0.5, 0.5, "데이터가 없습니다.", ha='center', va='center', transform=self.ax.transAxes)
            self.canvas.draw()
            return

        prices_ax = self.ax
        ratio_ax = prices_ax.twinx()

        # 시세 데이터 플로팅
        if sise_data:
            dates = [pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data]
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            if show_sise_sale:
                prices_ax.plot(dates, [d['매매일반거래가'] for d in sise_data], label='매매일반거래가', color='blue', linestyle='solid', linewidth=2, marker='None')
            if show_sise_lease:
                prices_ax.plot(dates, [d['전세일반거래가'] for d in sise_data], label='전세일반거래가', color='red', linestyle='solid', linewidth=2, marker='None')

            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')
            if show_sise_ratio:
                ratio_ax.plot(dates, [d['전세가율'] for d in sise_data], label='전세가율', color='green', linestyle=':', linewidth=2, marker='None')
        else:
            prices_ax.set_ylabel('가격 (만원)', color='blue')
            prices_ax.tick_params(axis='y', labelcolor='blue')
            ratio_ax.set_ylabel('전세가율 (%)', color='green')
            ratio_ax.tick_params(axis='y', labelcolor='green')

        # Y축 격자선 설정
        prices_ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, color='gray')
        prices_ax.grid(True, which='minor', axis='y', linestyle=':', linewidth=0.25, color='lightgray')

        # 실거래가 데이터 플로팅 (점도표)
        if show_real_sale and sale_real_data:
            sale_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None]
            sale_real_prices = [d['거래금액'] for d in sale_real_data if d['거래금액'] is not None]
            prices_ax.scatter(sale_real_dates, sale_real_prices, label='매매실거래', color='royalblue', marker='o', s=15, alpha=0.6)

        if show_real_lease and lease_real_data:
            lease_real_dates = [pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None]
            lease_real_prices = [d['거래금액'] for d in lease_real_data if d['거래금액'] is not None]
            prices_ax.scatter(lease_real_dates, lease_real_prices, label='전세실거래', color='firebrick', marker='s', s=15, alpha=0.6)

        # 하단 거래 건수 차트 그리기
        self.ax2.clear()
        bar_width = 15  # 막대 너비 (일 단위)

        sale_counts = {}
        lease_counts = {}

        if show_real_sale:
            for d in sale_real_data:
                month = d['기준년월일'][:6]
                sale_counts[month] = sale_counts.get(month, 0) + 1

        if show_real_lease:
            for d in lease_real_data:
                month = d['기준년월일'][:6]
                lease_counts[month] = lease_counts.get(month, 0) + 1

        months = sorted(set(list(sale_counts.keys()) + list(lease_counts.keys())))
        x = [pd.to_datetime(m, format='%Y%m') for m in months]
        sale_y = [sale_counts.get(m, 0) for m in months]
        lease_y = [lease_counts.get(m, 0) for m in months]

        if show_real_sale:
            self.ax2.bar([d - pd.Timedelta(days=bar_width/2) for d in x], sale_y, width=bar_width, color='royalblue', label='매매 건수')
        if show_real_lease:
            self.ax2.bar([d + pd.Timedelta(days=bar_width/2) for d in x], lease_y, width=bar_width, color='firebrick', label='전세 건수')

        self.ax2.set_ylabel('거래 건수')
        self.ax2.grid(True, which='major', axis='y')
        self.ax2.legend(loc='upper left')
        self.ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))

        # X축 날짜 포맷 설정
        all_dates = []
        if sise_data and (show_sise_sale or show_sise_lease or show_sise_ratio):
            all_dates.extend([pd.to_datetime(d['기준년월'], format='%Y%m') for d in sise_data])
        if sale_real_data and show_real_sale:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in sale_real_data if d['거래금액'] is not None])
        if lease_real_data and show_real_lease:
            all_dates.extend([pd.to_datetime(d['기준년월일'], format='%Y%m%d') for d in lease_real_data if d['거래금액'] is not None])

        if all_dates:
            min_date = min(all_dates) - pd.Timedelta(days=30)
            max_date = max(all_dates) + pd.Timedelta(days=30)
            self.ax.set_xlim(min_date, max_date)

            self.ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
            self.ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            self.ax.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')

            self.ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
            self.ax.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            self.ax2.grid(True, which='major', axis='x', linestyle='--', linewidth=0.5, color='gray')
            self.ax2.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.25, color='lightgray')

            plt.setp(self.ax.get_xticklabels(), visible=False)
            self.fig.autofmt_xdate(rotation=45)

        # 시세차트 범례
        lines_prices, labels_prices = prices_ax.get_legend_handles_labels()
        lines_ratio, labels_ratio = ratio_ax.get_legend_handles_labels()

        all_lines_for_ax = lines_prices + lines_ratio
        all_labels_for_ax = labels_prices + labels_ratio

        order_for_ax = ['매매일반거래가', '전세일반거래가', '전세가율', '매매실거래', '전세실거래']
        handles_labels_for_ax = []
        for h, l in zip(all_lines_for_ax, all_labels_for_ax):
            if l in order_for_ax:
                handles_labels_for_ax.append((h, l))

        handles_labels_for_ax.sort(key=lambda hl: order_for_ax.index(hl[1]))

        handles_ax, labels_ax = zip(*handles_labels_for_ax)
        self.ax.legend(handles_ax, labels_ax, loc='upper left')

        # 차트 제목 설정
        if apt_info:
            title_text = f"{apt_info.get('단지명', '단지명 없음')} - {apt_info.get('전용면적', '면적 없음')}㎡"
        else:
            title_text = "아파트 시세 및 실거래가 변화"
        self.ax.set_title(title_text)
        
        self.fig.tight_layout()
        self.canvas.draw()

    def update_chart_summary_info(self, chart_data):
        if not chart_data:
            # 정보 초기화
            for key in self.chart_info_labels:
                self.chart_info_labels[key].config(text=f"{key}: -")
            return

        sise_data = chart_data.get('sise_data', [])
        sale_real_data = chart_data.get('sale_real_data', [])
        lease_real_data = chart_data.get('lease_real_data', [])

        # 매매 시세 최고가/최근가
        max_sale_price_sise = None
        max_sale_price_sise_date = None
        latest_sale_price_sise = None
        if sise_data:
            latest_sale_price_sise = sise_data[-1].get('매매일반거래가')
            for item in sise_data:
                price = item.get('매매일반거래가')
                if price is not None:
                    if max_sale_price_sise is None or price > max_sale_price_sise:
                        max_sale_price_sise = price
                        max_sale_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m")
        self.chart_info_labels['매매최고가'].config(text=f"매매최고가: {max_sale_price_sise if max_sale_price_sise is not None else '-'} 만원 ({max_sale_price_sise_date.strftime('%Y-%m') if max_sale_price_sise_date else '-'})")
        self.chart_info_labels['매매최근가'].config(text=f"매매최근가: {latest_sale_price_sise if latest_sale_price_sise is not None else '-'} 만원")

        # 전세 시세 최고가/최근가
        max_lease_price_sise = None
        max_lease_price_sise_date = None
        latest_lease_price_sise = None
        if sise_data:
            latest_lease_price_sise = sise_data[-1].get('전세일반거래가')
            for item in sise_data:
                price = item.get('전세일반거래가')
                if price is not None:
                    if max_lease_price_sise is None or price > max_lease_price_sise:
                        max_lease_price_sise = price
                        max_lease_price_sise_date = datetime.strptime(item['기준년월'], "%Y%m")
        self.chart_info_labels['전세최고가'].config(text=f"전세최고가: {max_lease_price_sise if max_lease_price_sise is not None else '-'} 만원 ({max_lease_price_sise_date.strftime('%Y-%m') if max_lease_price_sise_date else '-'})")
        self.chart_info_labels['전세최근가'].config(text=f"전세최근가: {latest_lease_price_sise if latest_lease_price_sise is not None else '-'} 만원")

        # 실거래 매매 최고가/최근가
        max_sale_price_real = None
        max_sale_price_real_date = None
        latest_sale_price_real = None
        if sale_real_data:
            latest_sale_price_real = sale_real_data[-1].get('거래금액')
            for item in sale_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_sale_price_real is None or price > max_sale_price_real:
                        max_sale_price_real = price
                        max_sale_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래매매최고가'].config(text=f"실거래매매최고가: {max_sale_price_real if max_sale_price_real is not None else '-'} 만원 ({max_sale_price_real_date.strftime('%Y-%m-%d') if max_sale_price_real_date else '-'})")
        self.chart_info_labels['실거래매매최근가'].config(text=f"실거래매매최근가: {latest_sale_price_real if latest_sale_price_real is not None else '-'} 만원")

        # 실거래 전세 최고가/최근가
        max_lease_price_real = None
        max_lease_price_real_date = None
        latest_lease_price_real = None
        if lease_real_data:
            latest_lease_price_real = lease_real_data[-1].get('거래금액')
            for item in lease_real_data:
                price = item.get('거래금액')
                if price is not None:
                    if max_lease_price_real is None or price > max_lease_price_real:
                        max_lease_price_real = price
                        max_lease_price_real_date = datetime.strptime(item['기준년월일'], "%Y%m%d")
        self.chart_info_labels['실거래전세최고가'].config(text=f"실거래전세최고가: {max_lease_price_real if max_lease_price_real is not None else '-'} 만원 ({max_lease_price_real_date.strftime('%Y-%m-%d') if max_lease_price_real_date else '-'})")
        self.chart_info_labels['실거래전세최근가'].config(text=f"실거래전세최근가: {latest_lease_price_real if latest_lease_price_real is not None else '-'} 만원")

        # 매매하락가, 매매하락율, 매매회복율
        decline_price = None
        decline_rate_str = "N/A"
        recovery_rate_str = "N/A"

        if max_sale_price_sise is not None and max_sale_price_sise_date is not None and latest_sale_price_sise is not None:
            sise_after_peak_values = []
            for item in sise_data:
                item_date = datetime.strptime(item['기준년월'], "%Y%m")
                if item_date >= max_sale_price_sise_date and item.get('매매일반거래가') is not None:
                    sise_after_peak_values.append(item.get('매매일반거래가'))
            
            if sise_after_peak_values:
                decline_price = min(sise_after_peak_values)
                
                if max_sale_price_sise > 0:
                    decline_rate = ((max_sale_price_sise - decline_price) / max_sale_price_sise) * 100
                    decline_rate_str = f"{decline_rate:.2f}%"
                else:
                    decline_rate_str = "N/A"

                if decline_price is not None and max_sale_price_sise is not None and (max_sale_price_sise - decline_price) > 0:
                    if latest_sale_price_sise > decline_price and latest_sale_price_sise < max_sale_price_sise:
                        recovery_rate = ((latest_sale_price_sise - decline_price) / (max_sale_price_sise - decline_price)) * 100
                        recovery_rate_str = f"{recovery_rate:.2f}%"
                    elif latest_sale_price_sise >= max_sale_price_sise:
                        recovery_rate_str = "100.00% (최고가 회복)"
                    else:
                        recovery_rate_str = "0.00%"
                else:
                    recovery_rate_str = "N/A"
            else:
                decline_price = max_sale_price_sise
                decline_rate_str = "0.00%"
                recovery_rate_str = "N/A"
                
        self.chart_info_labels['매매하락가'].config(text=f"매매하락가: {decline_price if decline_price is not None else '-'} 만원")
        self.chart_info_labels['매매하락율'].config(text=f"매매하락율: {decline_rate_str}")
        self.chart_info_labels['매매회복율'].config(text=f"매매회복율: {recovery_rate_str}")

    def run(self):
        self.root.mainloop()


if __name__ == "__main__":
    app = InvestmentTableProgram()
    app.run()        

        